# Tabular Playground Series 7 - Predict Air Pollution

The dataset deals with predicting air pollution in a city via various input sensor values. The task is to predict, based on the sensor, values **three** target variables: target_carbon_monoxide,target_benzene and target_nitrogen_oxides. Submissions are evaluated using the [mean column-wise root mean squared logarithmic error](http://www.kaggle.com/c/tabular-playground-series-jul-2021/overview/evaluation).

The model is created based on my [EDA](https://www.kaggle.com/melanie7744/tps7-interactive-eda-with-plotly/edit).

Steps:
* feature engineering
* model tuning with Optuna
* visual checks on model performance
* predict & submit

At the end of the notebook there is a section about understanding the error metric.

Feedback is highly appreciated!


### I decided not to use the leak!


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sklearn
print("scikit-learn version: {}". format(sklearn.__version__))
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score

import matplotlib
import matplotlib.pyplot as plt
print("matplotlib version: {}". format(matplotlib.__version__))

import plotly
import plotly.express as px
import plotly.graph_objects as go
print("plotly version: {}". format(plotly.__version__))


import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance
print("xgboost version: {}". format(xgb.__version__))

import optuna

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# read input files
df_train = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv")
df_test = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv")
sample_submission = pd.read_csv("../input/tabular-playground-series-jul-2021/sample_submission.csv")
target_cols = ["target_carbon_monoxide","target_benzene","target_nitrogen_oxides"]
df_test = df_test.loc[1:] # remove the row that is duplicated in train and test, messes up lag, pct features. Is added again later.
df_all = df_train.drop(columns=target_cols).append(df_test) # makes preprocessing easier

In [ ]:
# store target columns
y_cm = df_train["target_carbon_monoxide"]
y_b = df_train["target_benzene"]
y_no = df_train["target_nitrogen_oxides"]
# store known target values
known_target = df_train.iloc[7110][target_cols]

## Adding features

In [ ]:

# consider daily sums here or 6h sums

# lags features
df_all = df_train.drop(columns=target_cols).append(df_test)
lag_columns = ['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5'] # define which for which columns shifts/lags shall be created
shift_range = [3,6,9,12] # define lag range

# create lags
for column in lag_columns:
    for i in shift_range:
        col_name = (column +"_lag_" + str(i))
        df_all[col_name] = df_all[column].shift(periods = i)
    
#df_all.head(10)


In [ ]:
# percentage change features, calculate change compared to previous hour, removed for now, did not improve result
#pct_columns = lag_columns
#for column in lag_columns:
#    col_name = column + "_pct"
#    df_all[col_name] = df_all[column].pct_change()*100
#df_all.head()

In [ ]:
df_all['date_time'] = pd.to_datetime(df_all['date_time'])
df_all['year'] = df_all['date_time'].dt.year
df_all['month'] = df_all['date_time'].dt.month
df_all['day'] = df_all['date_time'].dt.day
df_all['hour'] = df_all['date_time'].dt.hour
df_all['dayofweek'] = df_all['date_time'].dt.dayofweek
df_all['weekend'] = df_all['dayofweek'].apply(lambda x: 1 if (x>4)  else 0) # Sat, Sun are counted as weekend
df_all['rushhour'] = df_all['hour'].apply(lambda x: 1 if x in [8,9,18,19,20]  else 0) # define those hour as rush hour
df_all['time'] = df_all['date_time'].astype(np.int64)//10**9 # Unix time
#df_all.head()

In [ ]:
# check the rows where the test set starts 
df_all.iloc[7109:7113]

In [ ]:
# split again in train and test
df_train =  df_all.iloc[0:df_train.shape[0]]
df_test = df_all.iloc[df_train.shape[0]-1:]
# remove rows containing Nan due to the lags
df_train = df_train.loc[max(shift_range):]
y_no = y_no.loc[max(shift_range):]
y_cm = y_cm.loc[max(shift_range):]
y_b = y_b.loc[max(shift_range):]
#df_test.head()

## Tune Model
(Changed in Version 16.) Use 3 models to predict the 3 target variables. Use Optuna to tune the models. (No chained prediction for the moment any more.) Use k-fold cross validation. This is just a guess, seasonality captured in the features could be enough. And we don't have enough training data to make good use of seasonality.

**Step 1**: Setup Optuna Tuner

In [ ]:
kfolds = KFold(n_splits=5, shuffle=True, random_state=7)

In [ ]:
def tune(objective):
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100) 

    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score} \nOptimized parameters: {params}")
    return params

In [ ]:
def xgb_objective(trial):
    _n_estimators = trial.suggest_int("n_estimators", 50, 1000) 
    _max_depth = trial.suggest_int("max_depth", 2, 10) 
    _learning_rate = trial.suggest_float("learning_rate", 1e-6, 0.3, log=True)
    _gamma = trial.suggest_float("gamma", 0.01, 1)
    _min_child_weight = trial.suggest_int("min_child_weight", 1, 10)
    _subsample = trial.suggest_float('subsample', 0.5, 1)
    _colsample_bytree = trial.suggest_float('colsample_bytree', 0.7, 1)
    _reg_lambda = trial.suggest_float('reg_lambda', 0.01, 10, log=True)

    
    model = xgb.XGBRegressor(
        objective = 'reg:squarederror', #'reg:sqarederror' is default and gives better rsmle than 'reg:squaredlogerror'. why?
        n_estimators=_n_estimators,
        max_depth=_max_depth, 
        learning_rate=_learning_rate,
        gamma=_gamma,
        min_child_weight=_min_child_weight,
        subsample=_subsample,
        reg_lambda=_reg_lambda,
        random_state=7,
    )
    

    val_rmsle = []
    for train_index, test_index in kfolds.split(X):
 
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
        model.fit(
            X_train, 
            y_train, 
            eval_metric="rmsle", 
            eval_set=[(X_train, y_train), (X_val, y_val)], 
            verbose=False, 
            early_stopping_rounds = 50)
    
        val_rmsle.append(model.evals_result()['validation_1']['rmsle'][-1])
    
   
    score = sum(val_rmsle)/len(val_rmsle)
    
    return score


**Step 2:** Find hyperparameters for all three target variables seperately.

In [ ]:
X = df_train.drop(columns="date_time")
y = y_b
#xgb_params_y_b = tune(xgb_objective) # uncomment to run optuna tuning again, takes time....

# Optuna Result: Best score: 0.0863592 
xgb_params_y_b = {'n_estimators': 365,
 'max_depth': 10,
 'learning_rate': 0.02965812395223671,
 'gamma': 0.3086304942573926,
 'min_child_weight': 9.03951830237063,
 'subsample': 0.5054451890047076,
 'colsample_bytree': 0.9986600565238701,
 'reg_lambda': 1.5966168376624417}

In [ ]:
X = df_train.drop(columns="date_time")
y = y_cm
#xgb_params_y_cm = tune(xgb_objective) # uncomment to run optuna tuning again, takes time....

# Optuna Result: Best score: 0.09788859999999999 
xgb_params_y_cm = {'n_estimators': 418,
 'max_depth': 5,
 'learning_rate': 0.07432650387394699,
 'gamma': 0.0676479407897746,
 'min_child_weight': 1.3220415658077735,
 'subsample': 0.8146019010378812,
 'colsample_bytree': 0.8918164888978178,
 'reg_lambda': 5.394004764094763}

In [ ]:
X = df_train.drop(columns="date_time")
y = y_no
#xgb_params_y_no = tune(xgb_objective) # uncomment to run optuna tuning again, takes time....

#[W 2021-07-16 12:08:03,325] Trial 0 failed, because the objective function returned nan. --- todo: check what went wrong
# Optuna Result: Best score: 0.20224699999999998 
xgb_params_y_no = {'n_estimators': 434,
 'max_depth': 10,
 'learning_rate': 0.042338664081773426,
 'gamma': 0.43939165464469737,
 'min_child_weight': 2.6282895129902637,
 'subsample': 0.835280405845509,
 'colsample_bytree': 0.8120530400759991,
 'reg_lambda': 5.228573656907881}

## Visualize Model Performance with Validation Set

In [ ]:
# use all except 2 month as training set
# use the last 2 month weeks as validation set (methodisch nicht ganz sauber da diese Daten benutzt wurden um die Hyperparamter zu lernen, potential leakage)
X_train = df_train[(df_train.year == 2010) & (df_train.month.isin(np.arange(3, 11, 1)))].drop(columns="date_time")
len_train = X_train.shape[0] + max(shift_range) -1
y_train_no = y_no.loc[:len_train]
y_train_cm = y_cm.loc[:len_train]
y_train_b = y_b.loc[:len_train]

X_val = df_train.iloc[X_train.shape[0]:]
val_time = X_val.date_time
X_val = X_val.drop(columns="date_time")
y_val_no = y_no.loc[len_train+1:]
y_val_cm = y_cm.loc[len_train+1:]
y_val_b = y_b.loc[len_train+1:]

In [ ]:
# target_benzene
model = XGBRegressor(
    objective = 'reg:squarederror', # 'reg:squaredlogerror' gives worse rsmle results, is not able to fit to the peaks, change and recalculate viz to see that
    ** xgb_params_y_b,
    seed=7)

model.fit(
            X_train, 
            y_train_b, 
            eval_metric="rmsle", 
            eval_set=[(X_train, y_train_b), (X_val, y_val_b)], 
            verbose=10, 
            early_stopping_rounds = 10)
predictions_y_b = model.predict(X_val)

#[158]	validation_0-rmsle:0.05487	validation_1-rmsle:0.09711

In [ ]:
fig = go.Figure()

fig.add_trace(
      go.Scatter(x=val_time, 
                 y=y_val_b, 
                 mode = 'lines', 
                 line = {'color':'darkgoldenrod', 'width' : 1},
                 name="Benzene Ground Truth")
)
fig.add_trace(
      go.Scatter(x=val_time, 
                 y=predictions_y_b, 
                 mode = 'lines', 
                 line = {'color':'black', 'width' : 1},
                 name="Benzene Predicted")
)

fig.update_layout(
    title="Benzene Predicted (val data = last 2 month) vs. Ground Truth (train data)"
)
fig.show()

In [ ]:
# target_carbon_monoxides
model = XGBRegressor(
    objective = 'reg:squarederror', 
    ** xgb_params_y_cm,
    seed=7)

model.fit(
            X_train, 
            y_train_cm, 
            eval_metric="rmsle", 
            eval_set=[(X_train, y_train_cm), (X_val, y_val_cm)], 
            verbose=10, 
            early_stopping_rounds = 10)
predictions_y_cm = model.predict(X_val)

#[148]	validation_0-rmsle:0.06803	validation_1-rmsle:0.22248

In [ ]:
fig = go.Figure()

fig.add_trace(
      go.Scatter(x=val_time, 
                 y=y_val_cm, 
                 mode = 'lines', 
                 line = {'color':'darkgoldenrod', 'width' : 1},
                 name="Carbon Monoxide Ground Truth")
)
fig.add_trace(
      go.Scatter(x=val_time, 
                 y=predictions_y_cm, 
                 mode = 'lines', 
                 line = {'color':'black', 'width' : 1},
                 name="Carbon Monoxide Predicted")
)

fig.update_layout(
    title="Carbon Monoxide Predicted (val data = last 2 month) vs. Ground Truth (train data)"
)
fig.show()

In [ ]:
# target nitrogen_oxides
model = XGBRegressor(
    objective = 'reg:squarederror', # 'reg:squaredlogerror' gives worse rsmle results, is not able to fit to the data, change and recalculate viz to see that
    ** xgb_params_y_no,
    seed=7)

model.fit(
            X_train, 
            y_train_no, 
            eval_metric="rmsle", 
            eval_set=[(X_train, y_train_no), (X_val, y_val_no)], 
            verbose=30, 
            early_stopping_rounds = 10)
predictions_y_no = model.predict(X_val)

#[433]	validation_0-rmsle:0.04537	validation_1-rmsle:0.54051

In [ ]:
fig = go.Figure()

fig.add_trace(
      go.Scatter(x=val_time, 
                 y=y_val_no, 
                 mode = 'lines', 
                 line = {'color':'darkgoldenrod', 'width' : 1},
                 name="Nitrogen Oxides Ground Truth")
)
fig.add_trace(
      go.Scatter(x=val_time, 
                 y=predictions_y_no, 
                 mode = 'lines', 
                 line = {'color':'black', 'width' : 1},
                 name="Nitrogen Oxides Predicted")
)

fig.update_layout(
    title="Nitrogen Oxides Predicted (val data = last 2 month) vs. Ground Truth (train data)"
)
fig.show()

As already indicated by the RMSLE, the predictions for Nitrogen Oxides are worse than for the other 2 variables. The model is not able to capture the peeks well. It can be seen that in the timeframe from Dec 14th - Dec 17th where the sensor behave strange (see [EDA](https://www.kaggle.com/melanie7744/tps7-interactive-eda-with-plotly/edit) ), the predictions are worst. 

## Predict

Predictions are made for all three target variables seperately. Feature importance as seen by XGBoost is visualized.

In [ ]:
def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

In [ ]:
X_test = df_test.drop(columns="date_time")

In [ ]:
# target_benzene
X = df_train.drop(columns="date_time")
y = y_b

xgb_params_y_b = {'n_estimators': 365,
 'max_depth': 10,
 'learning_rate': 0.02965812395223671,
 'gamma': 0.3086304942573926,
 'min_child_weight': 9.03951830237063,
 'subsample': 0.5054451890047076,
 'colsample_bytree': 0.9986600565238701,
 'reg_lambda': 1.5966168376624417}

model = XGBRegressor(
    objective = 'reg:squarederror', # 'reg:squaredlogerror' gives worse rsmle results
    ** xgb_params_y_b,
    seed=7)

model.fit(X,y)
predictions_y_b = model.predict(X_test)

plot_features(model,(10,12) )
plt.show()

In [ ]:
# target_carbon_monoxide
X = df_train.drop(columns="date_time")
y = y_cm

xgb_params_y_cm = {'n_estimators': 418,
 'max_depth': 5,
 'learning_rate': 0.07432650387394699,
 'gamma': 0.0676479407897746,
 'min_child_weight': 1.3220415658077735,
 'subsample': 0.8146019010378812,
 'colsample_bytree': 0.8918164888978178,
 'reg_lambda': 5.394004764094763}

model = XGBRegressor(
    objective = 'reg:squarederror',
    ** xgb_params_y_cm,
    seed=7)

model.fit(X,y)
predictions_y_cm = model.predict(X_test)

plot_features(model,(10,12) )
plt.show()

In [ ]:
# target_nitrogen_oxides
X = df_train.drop(columns="date_time")
y = y_no

xgb_params_y_no = {'n_estimators': 434,
 'max_depth': 10,
 'learning_rate': 0.042338664081773426,
 'gamma': 0.43939165464469737,
 'min_child_weight': 2.6282895129902637,
 'subsample': 0.835280405845509,
 'colsample_bytree': 0.8120530400759991,
 'reg_lambda': 5.228573656907881}

model = XGBRegressor(
    objective = 'reg:squarederror', 
    ** xgb_params_y_no,
    seed=7)

model.fit(X,y)
predictions_y_no = model.predict(X_test)

plot_features(model,(10,12) )
plt.show()

## Submission

In [ ]:
# generate submission file (Public Score 0.20824)
submission_xgb = pd.DataFrame(data={"date_time" : sample_submission.date_time,
                               "target_carbon_monoxide" : predictions_y_cm,
                               "target_benzene" : predictions_y_b,
                               "target_nitrogen_oxides" : predictions_y_no})
submission_xgb.loc[0,target_cols] = known_target # replace the first row with the known values as timestamp is identical
submission_xgb.to_csv('submission_xgb.csv', index=False)
submission_xgb.head()

I generate multiple submission files using a factor to uplift my predictions. This idea is based on the graphic analysis using the validation set (predictions were often below ground truth) and the fact that RMSLE penalizes underpredictions harder than overpredictions.

After some unsucessfull trials with too strong factors, I found factors that actually improved my Public Score!

In [ ]:
# generate submission file #  very low factors (Public Score: 0.20777, improvement!)
submission_xgb = pd.DataFrame(data={"date_time" : sample_submission.date_time,
                               "target_carbon_monoxide" : predictions_y_cm*1.001,
                               "target_benzene" : predictions_y_b*1.001,
                               "target_nitrogen_oxides" : predictions_y_no*1.01})
submission_xgb.loc[0,target_cols] = known_target # replace the first row with the known values as timestamp is identical
submission_xgb.to_csv('submission_xgb_verylowfactors.csv', index=False)
submission_xgb.head()

In [ ]:
# generate submission file # low factors (gives Public score of 0.21209)
submission_xgb = pd.DataFrame(data={"date_time" : sample_submission.date_time,
                               "target_carbon_monoxide" : predictions_y_cm*1.05,
                               "target_benzene" : predictions_y_b*1.01,
                               "target_nitrogen_oxides" : predictions_y_no*1.1})
submission_xgb.loc[0,target_cols] = known_target # replace the first row with the known values as timestamp is identical
submission_xgb.to_csv('submission_xgb_low_factor.csv', index=False)
submission_xgb.head()

In [ ]:
# generate submission file # min factors (gives Public score of 0.20802)
submission_xgb = pd.DataFrame(data={"date_time" : sample_submission.date_time,
                               "target_carbon_monoxide" : predictions_y_cm*1.0007,
                               "target_benzene" : predictions_y_b*1.0007,
                               "target_nitrogen_oxides" : predictions_y_no*1.005})
submission_xgb.loc[0,target_cols] = known_target # replace the first row with the known values as timestamp is identical
submission_xgb.to_csv('submission_xgb_minfactor.csv', index=False)
submission_xgb.head()

In [ ]:
# generate submission file # new factors 
submission_xgb = pd.DataFrame(data={"date_time" : sample_submission.date_time,
                               "target_carbon_monoxide" : predictions_y_cm*1.001,
                               "target_benzene" : predictions_y_b*1.001,
                               "target_nitrogen_oxides" : predictions_y_no*1.005})
submission_xgb.loc[0,target_cols] = known_target # replace the first row with the known values as timestamp is identical
submission_xgb.to_csv('submission_xgb_newfactor.csv', index=False)

In [ ]:
# public score = 0.77258
sample_submission.head()

## Understanding the error metric

In [ ]:
p = 0.1
a = 0.2
((np.log(p+1) - np.log(a+1))**2)**0.5

In [ ]:
p = np.array(list(range(0,50,1)))/10 # predicted
a = np.array([1] * 50)            # actual
y=((np.log(p+1) - np.log(a+1))**2)**0.5 #rmsle
mse = (a-p)**2
rmse = ((a-p)**2)**0.5
#display(p)
#display(y)
#display(mse)
#display(rmse)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (12,6))
plt.plot(p,y, color = 'blue')
plt.vlines(1,-0.2,1.5, colors='green', label='actual value')
#plt.plot(p,mse, color = 'red')
plt.plot(p,rmse, color = 'black')
plt.xlabel("prediction")
plt.ylabel("rmsle")
plt.show()

Predictions lower than the ground truth are penalized harder than predictions above ground truth.

Compared to MSE and RMSE the RMSLE penalized predictions that are far off, less.

In [ ]:
from sklearn.metrics import mean_squared_log_error 
from sklearn.metrics import mean_squared_error

In [ ]:
y_true = [3, 2, 4]
y_pred = 1.1 * np.array(y_true)
print("MSLE:  ", mean_squared_log_error(y_true, y_pred))
print("RMLSE: ", np.sqrt(mean_squared_log_error(y_true, y_pred)))
print("RMLSE: ", mean_squared_error(np.log1p(y_true), np.log1p(y_pred), squared = False))

In [ ]:
p = np.array(y_pred)
a = np.array(y_true)
y=((np.log(p+1) - np.log(a+1))**2)
sum(y) / len(y)
print("MSLE:  ", sum(y) / len(y))
print("RMLSE: ", (sum(y) / len(y))**0.5)

In [ ]:
x = np.array(list(range(0,100)))/10
y = np.log(x+1)   # equivalent: y = np.log1p(x)

plt.figure(figsize = (12,6))
plt.plot(x,y)
plt.show()